In [ ]:
import ipywidgets as widgets
from IPython.display import display
import json
from datetime import datetime

todo_items = []

def progress_to_color(progress):
    level = int(255 * (1 - progress / 100))
    return f"#{level:02x}{level:02x}{level:02x}"

layout_full = widgets.Layout(width="100%")
layout_medium = widgets.Layout(width="160px")
layout_label = widgets.Layout(width="80px")
layout_button = widgets.Layout(width="160px", height="36px")
layout_progress_bar = widgets.Layout(width="100%", height="20px")

overall_progress_bar = widgets.FloatProgress(value=0, min=0, max=100, layout=layout_progress_bar)
overall_progress_label = widgets.Label(value="(0%)")

todo_list_box = widgets.VBox()

def update_overall_progress():
    if not todo_items:
        overall_progress_bar.value = 0
        overall_progress_bar.style.bar_color = "#ffffff"
        overall_progress_label.value = "(0%)"
        return
    avg = sum(i['progress'] for i in todo_items) / len(todo_items)
    overall_progress_bar.value = avg
    overall_progress_bar.style.bar_color = progress_to_color(avg)
    overall_progress_label.value = f"({int(avg)}%)"

def refresh_list():
    todo_list_box.children = []
    for item in todo_items:
        title = widgets.Text(value=item['title'], layout=layout_medium)
        progress_bar = widgets.FloatProgress(value=item['progress'], min=0, max=100, layout=layout_progress_bar)
        progress_bar.style.bar_color = progress_to_color(item['progress'])
        progress_slider = widgets.IntSlider(value=item['progress'], min=0, max=100, step=5, layout=layout_full)

        def on_progress_change(change, i=item, bar=progress_bar):
            i['progress'] = change['new']
            bar.value = change['new']
            bar.style.bar_color = progress_to_color(change['new'])
            update_overall_progress()
        progress_slider.observe(on_progress_change, names='value')

        due_label = widgets.Label(value=item['due'].strftime('%Y-%m-%d') if item['due'] else "-", layout=layout_medium)
        priority_label = widgets.Label(value=item['priority'], layout=layout_medium)

        complete_btn = widgets.Button(description="완료✅", button_style="success", layout=layout_button)
        delete_btn = widgets.Button(description="삭제❌", button_style="danger", layout=layout_button)

        def on_complete(b, i=item):
            i['progress'] = 100
            refresh_list()
            update_overall_progress()
        def on_delete(b, i=item):
            todo_items.remove(i)
            refresh_list()
            update_overall_progress()
        complete_btn.on_click(on_complete)
        delete_btn.on_click(on_delete)

        row = widgets.HBox([
            widgets.VBox([
                widgets.HBox([widgets.Label("할 일", layout=layout_label), title]),
                widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_bar]),
                widgets.HBox([widgets.Label("", layout=layout_label), progress_slider]),
                widgets.HBox([widgets.Label("마감일", layout=layout_label), due_label]),
                widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_label]),
                widgets.HBox([widgets.Label("작업", layout=layout_label), widgets.VBox([complete_btn, delete_btn])])
            ])
        ], layout=layout_full)
        todo_list_box.children = list(todo_list_box.children) + [row]

title_input = widgets.Text(placeholder="할 일을 입력하세요", layout=layout_medium)
progress_input = widgets.IntSlider(value=0, min=0, max=100, step=5, layout=layout_full)
due_input = widgets.DatePicker(layout=layout_medium)
priority_input = widgets.Dropdown(options=["낮음", "보통", "높음"], value="보통", layout=layout_medium)

add_btn = widgets.Button(description="추가", button_style="success", layout=widgets.Layout(width="340px", height="40px"))
def add_task(b):
    if title_input.value.strip():
        todo_items.append({
            "title": title_input.value.strip(),
            "progress": progress_input.value,
            "due": due_input.value,
            "priority": priority_input.value
        })
        title_input.value = ""
        progress_input.value = 0
        due_input.value = None
        priority_input.value = "보통"
        update_overall_progress()
        refresh_list()
add_btn.on_click(add_task)

def save_data(b):
    with open("todo_data.json", "w") as f:
        json.dump([{
            **i,
            "due": i["due"].strftime('%Y-%m-%d') if i["due"] else None
        } for i in todo_items], f)

def load_data(b):
    try:
        with open("todo_data.json", "r") as f:
            raw = json.load(f)
            todo_items.clear()
            for i in raw:
                todo_items.append({
                    "title": i["title"],
                    "progress": i["progress"],
                    "due": datetime.strptime(i["due"], "%Y-%m-%d").date() if i["due"] else None,
                    "priority": i["priority"]
                })
            update_overall_progress()
            refresh_list()
    except Exception as e:
        print("불러오기 실패:", e)

save_btn = widgets.Button(description="저장", button_style="info", layout=layout_button)
load_btn = widgets.Button(description="불러오기", button_style="warning", layout=layout_button)
save_btn.on_click(save_data)
load_btn.on_click(load_data)

app = widgets.VBox([
    widgets.HTML("<h3>DAVID mk24 - 회색 → 검정 진행률 및 레이아웃 정리</h3>"),
    widgets.HBox([widgets.Label("할 일", layout=layout_label), title_input]),
    widgets.HBox([widgets.Label("진행률", layout=layout_label), progress_input]),
    widgets.HBox([widgets.Label("마감일", layout=layout_label), due_input]),
    widgets.HBox([widgets.Label("중요도", layout=layout_label), priority_input]),
    widgets.HBox([add_btn]),
    widgets.HBox([save_btn, load_btn]),
    widgets.HBox([overall_progress_bar, overall_progress_label]),
    todo_list_box
])

display(app)
update_overall_progress()
refresh_list()